# Example from section 3.6.2
This notebook gives the results presented in Table 4 of the paper *Optimal monotonicity–preserving perturbations of a given Runge-Kutta method*.

In [ ]:
%matplotlib inline
from nodepy import rk
import numpy as np

In [ ]:
N = 20
x = np.linspace(0,1.,N)
dx = x[1]-x[0]
a = lambda xx, tt : (np.cos(200*xx + 400*tt))**4

In [ ]:
def updiff_matrix(t):
    "First order upwind flux difference with variable velocity"
    N = len(x)
    A = np.zeros((N,N))
    for i in xrange(N):
        A[i,i]   = - a(x[i],t)
        A[i,i-1] =   a(x[i-1],t)
    A[0,-1] = 0.
    return 1./dx * A

def downdiff_matrix(t):
    "First order downwind flux difference with variable velocity"
    N = len(x)
    A = np.zeros((N,N))
    for i in xrange(N-1):
        A[i,i+1] =   a(x[i+1],t)
        A[i,i]   = - a(x[i],t)
    A[N-1,N-1] = - a(x[N-1],t)
    return 1./dx * A

def propagation_matrix(rkm,h):
    """Constructs the effective matrix M where u_{n+1} = M u_n.
       This version uses Kronecker products."""
    N = len(x)
    nstage = len(rkm)
    I = np.eye(nstage)
    I2 = np.eye(N)
    Z = np.zeros((nstage*N,nstage*N))
    for i in xrange(len(rkm)):
        Z[i*N:(i+1)*N,i*N:(i+1)*N] = h*updiff_matrix(h*rkm.c[i])
    X=np.kron(I,I2)-np.dot(np.kron(rkm.A,I2),Z)
    Xinv = np.linalg.inv(X)
    e = np.kron(np.ones(nstage)[:,np.newaxis],I2)
    M = I2 + np.dot(np.kron(rkm.b[:,np.newaxis],I2).T,
                  np.dot(Z,np.dot(Xinv,e)))
    return M

In [ ]:
method = rk.loadRKM('RK44')
method = method.__num__()
r, v, alphaup, alphadown = method.optimal_perturbed_splitting()
alphaup = np.array(alphaup,dtype='float64')
alphadown = np.array(alphadown,dtype='float64')

In [ ]:
def propagation_matrix_manual(rkm,h):
    """Constructs the effective matrix M where u_{n+1} = M u_n.
       This version does it the naive way."""
    N = len(x)
    nstage = len(rkm)
    M_stage = []
    for i in range(nstage):
        Mi = np.eye(N)
        for j in range(i):
            Mi += rkm.A[i,j]*h*np.dot(updiff_matrix(h*rkm.c[j]),M_stage[j])
        M_stage.append(Mi)
    M = np.eye(N)
    for j in range(nstage):
        M += rkm.b[j] * h * np.dot(updiff_matrix(h*rkm.c[j]),M_stage[j])
    return M

In [ ]:
cflnum = 0.17
dt = cflnum*dx

M = propagation_matrix(method,dt)
print M.min()
print np.linalg.norm(M,1)

M = propagation_matrix_manual(method,dt)
print M.min()
print np.linalg.norm(M,1)

In [ ]:
cflnum = 0.18
dt = cflnum*dx

M = propagation_matrix(method,dt)
print M.min()
print np.linalg.norm(M,1)

M = propagation_matrix_manual(method,dt)
print M.min()
print np.linalg.norm(M,1)

In [ ]:
def propagation_matrix_manual_downwind_canonical(r,v,alphaup,alphadown,c,h):
    """Constructs the effective matrix M where u_{n+1} = M u_n,
       for perturbed methods in canonical form.  This version does it the naive way."""
    N = len(x)
    nstage = len(v) - 1
    M_stage = []
    I = np.eye(N)
    for i in range(nstage+1):
        Mi = v[i]*I
        for j in range(i):
            Mi += alphaup[i,j]*np.dot(I+h/r*updiff_matrix(h*c[j]),M_stage[j])
            Mi += alphadown[i,j]*np.dot(I+h/r*downdiff_matrix(h*c[j]),M_stage[j])
        M_stage.append(Mi)
    return Mi

In [ ]:
cflnum = 0.685
dt = cflnum*dx
alphaup = alphaup.astype('float64')
alphadown = alphadown.astype('float64')
v = v.astype('float64')
M = propagation_matrix_manual_downwind_canonical(r,v,alphaup,alphadown,method.c.astype(float),dt)
print M.min()
print np.linalg.norm(M,1)

cflnum = 0.686
dt = cflnum*dx

M = propagation_matrix_manual_downwind_canonical(r,v,alphaup,alphadown,method.c.astype(float),dt)
print M.min()
print np.linalg.norm(M,1)

In [ ]:
def find_max_positive_stepsize(method,epsilon=1.e-12):
    method = method.__num__()
    
    r, v, alphaup, alphadown = method.optimal_perturbed_splitting()
    alphaup = alphaup.astype('float64')
    alphadown = alphadown.astype('float64')
    v = v.astype('float64')

    cflnum = 0.0
    while True:
        cflnum += 0.01
        dt = cflnum * dx
        M = propagation_matrix(method,dt)
        if M.min() < -epsilon:
            break
        if np.linalg.norm(M,1) > 1 + epsilon:
            break
    
    unperturbed = cflnum - 0.01
    
    cflnum = 0.0
    while True:
        cflnum += 0.01
        dt = cflnum * dx
        M = propagation_matrix_manual_downwind_canonical(r,v,alphaup,alphadown,method.c.astype('float64'),dt)
        if M.min() < -epsilon:
            #print M.min()
            break
        if np.linalg.norm(M,1) > 1 + epsilon:
            #print np.linalg.norm(M,1)-1
            break
    
    perturbed = cflnum - 0.01
    return unperturbed, perturbed

In [ ]:
for name in ('FE','Mid22','MTE22','SSP22','SSP22star','Heun33','SSP33','RK44',
             'Merson43','SSP104','Fehlberg45','DP5','BS5','SSP75','SSP85','SSP95',
             'CMR6','PD8'):
    method = rk.loadRKM(name)
    u, p = find_max_positive_stepsize(method)
    print name, u, p